In [1]:
import pyvisa as visa
import time
import numpy as np
import pymeasure
from pymeasure.instruments.keithley import Keithley2400
import pandas as pd
import matplotlib.pyplot as plt
import datetime

#Check 6221 compliance, Check 2400 output off state

In [2]:
import datetime
print(datetime.datetime.now())

2021-10-02 00:40:15.551614


In [3]:
import os

In [4]:
print(os.getcwd())

C:\Research\Projects\Sam files\_switch


In [5]:
rm = visa.ResourceManager()
rm.list_resources()

('GPIB0::1::INSTR',
 'GPIB0::6::INSTR',
 'GPIB0::7::INSTR',
 'GPIB0::9::INSTR',
 'GPIB0::12::INSTR',
 'GPIB0::22::INSTR',
 'GPIB0::23::INSTR')

In [14]:
sensitivity = {1:0, 0.1:3, 0.01:6, 0.001:9, 0.5:1, 0.00005:13,
               0.0001:12, 0.00001:15, 0.000001:18, 0.0002:11, 0.05:4, 0.00002:14}
import pyvisa as visa
class Lockin:
    def __init__(self, resource, gpib=1):
        self.lock_in = resource.open_resource('GPIB0::{}::INSTR'.format(gpib))

    def change_harmonic(self, harm=1, sens=1):
        self.lock_in.write("HARM {}".format(harm))
        self.lock_in.write("SCAL {}".format(sensitivity[sens]))

    def lock_in_read(self):
        x = float(self.lock_in.query("OUTP? 1").strip())
        y = float(self.lock_in.query("OUTP? 2").strip())
        r = float(self.lock_in.query("OUTP? 3").strip())
        theta = float(self.lock_in.query("OUTP? 4").strip())
        return x, y, r, theta

    def lock_in_measure(self, count=10, time_step=0.1,
                        wait_before_measure=3,
                        harm=2, sens=0.00001):
        self.change_harmonic(harm, sens)
        time.sleep(wait_before_measure)
        xs, ys, rs, thetas = [], [], [], []
        for i in range(count):
            x, y, r, theta = self.lock_in_read()
            xs += [x]
            ys += [y]
            rs += [r]
            thetas += [theta]
            time.sleep(time_step)
        x_mean, x_std = np.mean(xs), np.std(xs)
        y_mean, y_std = np.mean(ys), np.std(ys)
        r_mean, r_std = np.mean(rs), np.std(rs)
        theta_mean, theta_std = np.mean(thetas), np.std(thetas)

        return x_mean, y_mean, r_mean, theta_mean, \
               x_std, y_std,r_std, theta_std

In [15]:
lock = Lockin(rm, 1)

In [18]:
lock.lock_in_read()

(1.38283e-05, -9.23878e-06, 3.87433e-06, 84.6)

In [73]:
lock.change_harmonic(1,0.0001)

In [45]:
#hall measurement connection, chan 1 rxy, chan 2 rxx
dc1_ID=24
dc2_ID=30
volt1_ID=9
volt2_ID=7
lockin_ID=1
dc1 = Keithley2400("GPIB::{}".format(dc1_ID))
dc2 = Keithley2400("GPIB::{}".format(dc2_ID))
volt1 = rm.open_resource('GPIB0::{}::INSTR'.format(volt1_ID))
volt2 = rm.open_resource('GPIB0::{}::INSTR'.format(volt2_ID))
lockin1=rm.open_resource('GPIB0::{}::INSTR'.format(lockin_ID))

In [10]:
volt2.write("*RST")

(6, <StatusCode.success: 0>)

In [3]:
#switch
ac_6221_1_id = 14
ac_6221__2id = 3
#ac_6221__3id = 12 #mid
dc_2400_id = 30
nanovolt_id = 8

ac_1 = rm.open_resource('GPIB0::{}::INSTR'.format(ac_6221_1_id))
ac_2 = rm.open_resource('GPIB0::{}::INSTR'.format(ac_6221__2id))
#ac_mid = rm.open_resource('GPIB0::{}::INSTR'.format(ac_6221__3id))
dc = Keithley2400("GPIB::{}".format(dc_2400_id))
volt = rm.open_resource('GPIB0::{}::INSTR'.format(nanovolt_id))

In [7]:
dc1 = dc
volt1 = volt

In [80]:
volt2.write("*RST")

(6, <StatusCode.success: 0>)

In [6]:
def nano_change_chanel(volt, chan=None):
    if not chan:
        return
    volt.write(":SENS:CHAN {}".format(chan))
    time.sleep(0.1)
    if volt.query(":SENS:CHAN?").strip() == str(chan):
        return
    else:
        print("channel switch error")
        raise

In [ ]:
volt2.write(":sens:volt:chan2:rang 0.8")

In [32]:
nano_change_chanel(volt1, chan=2)

In [73]:
def init_hall_two_channel(dc_current=50e-6, step=3):
    for i in range(step):
        print("sample 1:" + str(i) + "chan1, std_1, chan_2,std_2")
        print(measure_resistance_two_channel(dc1, volt1, dc_current=dc_current))
    print("-"*10)
    for i in range(step):
        print("sample 2:" + str(i) + "chan1, std_1, chan_2,std_2")
        print(measure_resistance_two_channel(dc2, volt2, dc_current=dc_current))

In [153]:
init_hall_two_channel(step=2)

sample 1:0chan1, std_1, chan_2,std_2
(1.529657879642857e-05, 3.578105817120136e-07, -0.033841671689285716, 1.0761184490576784e-06)
sample 1:1chan1, std_1, chan_2,std_2
(1.513360055e-05, 7.725756071797704e-08, -0.03384028545714286, 1.0309298687209962e-06)
sample 2:0chan1, std_1, chan_2,std_2
(1.5800393471428573e-05, 1.1934627527690916e-08, -0.03158743318571428, 2.8296707814529716e-07)
sample 2:1chan1, std_1, chan_2,std_2
(1.5805957321428573e-05, 6.5240145410214705e-09, -0.03158916902857142, 4.725347623991482e-07)


In [11]:
def init_hall_communication(dc_I_1=100e-6,dc_I_2=100e-6):
    if dc_I_1:
        dc1.apply_current()# Sets up to source current
        dc1.source_current_range = 10e-3   # Sets the source current range to 10 mA
        dc1.compliance_voltage = 10        # Sets the compliance voltage to 10 V
        dc1.source_current = dc_I_1          # Sets the source current to 0.1 mA
        dc1.enable_source() 
        #print("enable dc1 current now, current:" + str(dc))
        print("check nano volt 1:")
        for i in range(10):
            print(volt1.query("fetch?"))
            time.sleep(0.1)
        time.sleep(10)
        dc1.shutdown()   
        print("dc1 current disabled")
    
    if dc_I_2:
        dc2.apply_current()# Sets up to source current
        dc2.source_current_range = 10e-3   # Sets the source current range to 10 mA
        dc2.compliance_voltage = 10        # Sets the compliance voltage to 10 V
        dc2.source_current = dc_I_2           # Sets the source current to 0.1 mA
        dc2.enable_source() 
        #print("enable dc2 current now, current:" + str(dc))
        print("check nano volt 2:")
        for i in range(10):
            print(volt2.query("fetch?"))
            time.sleep(0.1)
        time.sleep(10)
        dc2.shutdown()   
        print("dc2 current disabled")


In [75]:
init_hall_communication(dc_I_1=100e-6,dc_I_2=100e-6)

check nano volt 1:
-8.56870275E+00

-8.57147729E+00

-8.57147729E+00

-9.61726012E+00

-9.61726012E+00

-9.65703449E+00

-9.65703449E+00

-9.70296605E+00

-9.70296605E+00

-9.68581406E+00

dc1 current disabled
check nano volt 2:
-1.97505871E-06

-1.97505871E-06

-1.21166125E-06

-1.21166125E-06

-1.21166125E-06

+2.12067189E-05

+2.12067189E-05

+2.12067189E-05

+2.11675393E-05

+2.11675393E-05

dc2 current disabled


In [117]:
init_hall_communication(dc_I_1=100e-6,dc_I_2=-100e-6)

check nano volt 1:
+9.9E37

+9.9E37

+9.9E37

+9.9E37

+9.9E37

+9.9E37

+9.9E37

+9.9E37

+9.9E37

+9.9E37

dc1 current disabled
check nano volt 2:
+1.20270061E-06

+1.20270061E-06

+1.20270061E-06

+2.55008444E-06

+2.55008444E-06

+2.55008444E-06

+8.21760558E-06

+8.21760558E-06

+8.21760558E-06

+8.26495471E-06

dc2 current disabled


In [4]:
def trigger_ac(ac, offs=0.005, amp=1e-6, duration=0.005, wait_after_arm=2):
    ac.write("SOUR:WAVE:ABOR")
    ac.write("SOUR:WAVE:OFFS {}".format(offs))
    ac.write("SOUR:WAVE:AMPL {}".format(amp))
    ac.write("SOUR:WAVE:DUR:TIME {}".format(duration))
    ac.write("SOUR:WAVE:ARM")
    time.sleep(wait_after_arm)
    ac.write("SOUR:WAVE:INIT")

In [5]:
def init_communication(amp=0.00005):
    print("check ac_1: offerset 50uA, amp1uA, duration 10s")
    trigger_ac(ac_1, offs=amp, amp=1e-6, duration=5, wait_after_arm=2)
    time.sleep(15)
    
    print("check ac_2: offerset 50uA, amp1uA, duration 10s")
    trigger_ac(ac_2, offs=amp, amp=1e-6, duration=5, wait_after_arm=2)
    time.sleep(15)
    """
    print("check ac_mid: offerset 50uA, amp1uA, duration 10s")
    trigger_ac(ac_mid, offs=amp, amp=1e-6, duration=5, wait_after_arm=2)
    time.sleep(15)"""
    
    dc.apply_current()# Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    dc.compliance_voltage = 10        # Sets the compliance voltage to 10 V
    dc.source_current = amp            # Sets the source current to 0.1 mA
    dc.enable_source() 
    print("enable dc current now, current:" + str(dc))
    print("check nano volt:")
    for i in range(10):
        print(volt.query("fetch?"))
        time.sleep(0.1)
    time.sleep(10)
    dc.shutdown()   
    print("dc current disabled")


In [34]:
init_communication(amp=1e-4)

check ac_1: offerset 50uA, amp1uA, duration 10s
check ac_2: offerset 50uA, amp1uA, duration 10s
enable dc current now, current:<pymeasure.instruments.keithley.keithley2400.Keithley2400 object at 0x000001C1350F48E0>
check nano volt:
+3.72603510E-07

+3.72603510E-07

+3.72603510E-07

-1.03604106E-05

-1.03604106E-05

-1.03604106E-05

-1.03604106E-05

-1.99992209E-05

-1.99992209E-05

-1.99992209E-05

dc current disabled


In [7]:
def measure_resistance(dc, volt, dc_current=100e-6, measure_points=30, skip_points=10, time_per_point=0.1):
    dc.apply_current()  # Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    dc.compliance_voltage = 10        # Sets the compliance voltage to 10 V
    dc.source_current = dc_current            # Sets the source current to 0 mA
    dc.enable_source() 
    v_up = []
    for i in range(30):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up.append(float(volt.query("fetch?")))
    v_up = sorted(v_up)
    v_up = v_up[3: -3]
    # Sets the compliance voltage to 10 V
    dc.source_current = -1 * dc_current
    dc.enable_source() 
    
    v_down = []
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down.append(float(volt.query("fetch?")))
    
    dc.shutdown() 
    #dc.source_current = dc_current
    #dc.enable_source() 
    
    v_down = sorted(v_down)
    v_down = v_down[3: -3]
    
    average_v = (np.array(v_up).mean() - np.array(v_down).mean())/2
    std_v = (np.array(v_up).std() + np.array(v_down).std())/2
    return average_v, std_v

In [8]:
import subprocess
def get_position():
    result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_position.py'])

    current_p = round(float(result.decode('utf-8').strip().split(',')[1][:-1]))
    return current_p

In [9]:
get_position()

345

In [38]:
"""def measure_resistance_two_channel(dc, volt, dc_current):
    nano_change_chanel(volt, chan=1)
    average_v_1, std_v_1 = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
    nano_change_chanel(volt, chan=2)
    average_v_2, std_v_2 = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
    return average_v_1, std_v_1, average_v_2, std_v_2"""

'def measure_resistance_two_channel(dc, volt, dc_current):\n    nano_change_chanel(volt, chan=1)\n    average_v_1, std_v_1 = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)\n    nano_change_chanel(volt, chan=2)\n    average_v_2, std_v_2 = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)\n    return average_v_1, std_v_1, average_v_2, std_v_2'

In [54]:
nano_change_chanel(volt2, chan=1)

In [146]:
def measure_resistance_two_channel(dc, volt, dc_current=100e-6, measure_points=30, skip_points=10, time_per_point=0.05):
    #dc.apply_current()  # Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    
    dc.compliance_voltage = 100        # Sets the compliance voltage to 100 V
    
    dc.source_current = dc_current            # Sets the source current to 0 mA
    dc.enable_source() 
    v_up_1 = []
    nano_change_chanel(volt, chan=1)
    for i in range(measure_points):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up_1.append(float(volt.query("fetch?")))
    v_up_1 = sorted(v_up_1)
    v_up_1 = v_up_1[3: -3]
    
    
    v_up_2 = []
    nano_change_chanel(volt, chan=2)
    for i in range(measure_points):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up_2.append(float(volt.query("fetch?")))
    v_up_2 = sorted(v_up_2)
    v_up_2 = v_up_2[3: -3]
    
    # Sets the compliance voltage to 10 V
    dc.source_current = -1 * dc_current
    dc.enable_source() 
    
    v_down_1 = []
    nano_change_chanel(volt, chan=1)
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down_1.append(float(volt.query("fetch?")))
        
        
    v_down_2 = []
    nano_change_chanel(volt, chan=2)
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down_2.append(float(volt.query("fetch?")))
    
    dc.shutdown() 

    
    v_down_1 = sorted(v_down_1)
    v_down_1 = v_down_1[3: -3]
    v_down_2 = sorted(v_down_2)
    v_down_2 = v_down_2[3: -3]
    
    average_v_1 = (np.array(v_up_1).mean() - np.array(v_down_1).mean())/2
    std_v_1 = (np.array(v_up_1).std() + np.array(v_down_1).std())/2
    
    average_v_2 = (np.array(v_up_2).mean() - np.array(v_down_2).mean())/2
    std_v_2 = (np.array(v_up_2).std() + np.array(v_down_2).std())/2
    return average_v_1, std_v_1, average_v_2, std_v_2

In [152]:
i = [1e-6, 3e-6, 1e-5, 3e-5, 1e-4, 3e-4, 1e-3, 3e-3, 1e-2]
i = i + i[::-1]
file_name = "./_switch/data/XD346_5nm_Cr2O3_nag_1T_250K_current_dependence.csv"
with open(file_name, 'w') as f:
    f.write("v,std_v,v_2,std_2,I\n")
for c in i:
    time.sleep(10)
    res =  measure_resistance_two_channel(dc2, volt2, dc_current=c)
    res = list(res)
    res.append(c)
    with open(file_name, 'a') as f:
        res = [str(i) for i in res]
        f.write(",".join(res)+ "\n")
   

In [150]:
set_Field(-10000)

-10000.1201172
current field -10000.1201172


In [147]:
measure_resistance_two_channel(dc1, volt1, dc_current=0.05e-3)

(1.5235177989285713e-05,
 4.5353361202287543e-08,
 -0.03384192092142857,
 1.2644607012447607e-06)

In [290]:
def measure_resistance_with_off_set(ac=ac_1, volt=volt, dc_current=100e-6, measure_points=30, skip_points=10, time_per_point=0.1, duration=4):
    trigger_ac(ac, offs=dc_current, amp=1e-6, duration=duration, wait_after_arm=0)
    v_up = []
    for i in range(30):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up.append(float(volt.query("fetch?")))
    v_up = sorted(v_up)
    v_up = v_up[3: -3]
    # Sets the compliance voltage to 10 V
    time.sleep(duration - 2)
    
    trigger_ac(ac, offs=-1 * dc_current, amp=1 * 1e-6, duration=duration, wait_after_arm=0)
    
    v_down = []
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down.append(float(volt.query("fetch?")))
    
    dc.shutdown() 
    #dc.source_current = dc_current
    #dc.enable_source() 
    
    v_down = sorted(v_down)
    v_down = v_down[3: -3]
    
    average_v = (np.array(v_up).mean() - np.array(v_down).mean())/2
    std_v = (np.array(v_up).std() + np.array(v_down).std())/2
    return average_v, std_v

In [16]:
def do_measurement(sample_id='', ac_current=5*1e-3, dc_current=100e-6, duration=0.005, round=6, point_per_round=5, wait_after_trigger=15):
    file_name = "./_switch/data/{}ac_{}uA{}ms_wait{}s_{}.csv".format(sample_id, int(ac_current * 1e6), 
                                                                     int(duration * 1e3), wait_after_trigger, int(time.time()))
    with open(file_name, 'w') as f:
        f.write("round,ac_source,point_count,average_v, std_v, dc_current\n")

    for i in range(round):
        print("round: " + str(i+1))
        #for current in ['ac_2','ac_1']:
        for current in ['ac_1', 'ac_2']:
            print("current:" + current)
            for point in range(point_per_round):
                if current == 'ac_1':
                    trigger_ac(ac_1, offs=ac_current, duration=duration)

                else:
                    trigger_ac(ac_2, offs=ac_current, duration=duration)
                time.sleep(wait_after_trigger)
                average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
                with open(file_name, 'a') as f:
                    f.write("{},{},{},{},{},{}\n".format(i, current, point, average_v, std_v, dc_current))
                print(average_v)
                time.sleep(20)
                """
                plt.clf()
                df = pd.read_csv(file_name)
                v = df['average_v']
                v.plot()
                plt.show()"""

In [20]:
def do_measurement_three_pulses(sample_id='', ac_current=5*1e-3, dc_current=100e-6, duration=0.005, round=6, point_per_round=5,
                                wait_after_trigger=15, pulse=None):
    file_name = "data/{}ac_{}uA{}ms_wait{}s_{}.csv".format(sample_id, int(ac_current * 1e6), 
                                                                     int(duration * 1e3), wait_after_trigger, int(time.time()))
    if not pulse is None:
        file_name = file_name.replace(".csv", "".join(pulse) + ".csv")
    with open(file_name, 'w') as f:
        f.write("round,ac_source,point_count,average_v, std_v, dc_current\n")
        
        
    average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
    with open(file_name, 'a') as f:
        f.write("{},{},{},{},{},{}\n".format(0, 'init', 0, average_v, std_v, dc_current))

    for i in range(round):
        print("round: " + str(i+1))
        if pulse is None:
            pulse = ['ac_mid', '-ac_mid', 'ac_2', '-ac_2', 'ac_mid', '-ac_mid', 'ac_1', '-ac_1']
        for current in pulse:
            print("current:" + current)
            for point in range(point_per_round):
                if current == 'ac_1':
                    trigger_ac(ac_1, offs=ac_current, duration=duration)
                elif current == '-ac_1':
                    trigger_ac(ac_1, offs=-1 * ac_current, duration=duration)
                elif current == 'ac_mid':
                    trigger_ac(ac_mid, offs=ac_current, duration=duration)
                elif current == '-ac_mid':
                    trigger_ac(ac_mid, offs=-1 * ac_current, duration=duration)
                elif current == 'ac_2':
                    trigger_ac(ac_2, offs=ac_current, duration=duration)
                elif current == '-ac_2':
                    trigger_ac(ac_2, offs=-1 * ac_current, duration=duration)
                else:
                    print("no ac " + current)
                    raise
                    
                time.sleep(wait_after_trigger)
                average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
                with open(file_name, 'a') as f:
                    f.write("{},{},{},{},{},{}\n".format(i, current, point, average_v, std_v, dc_current))
                print(average_v)
                time.sleep(3)
                """
                plt.clf()
                df = pd.read_csv(file_name)
                v = df['average_v']
                v.plot()
                plt.show()"""

In [11]:
def do_measurement_with_off_set(sample_id='', ac_current=5*1e-3, dc_current=100e-6, duration=0.005, 
                                round=6, point_per_round=5, wait_after_trigger=15):
    file_name = "./_switch/data/{}ac_{}uA{}ms_wait{}s_{}.csv".format(sample_id, int(ac_current * 1e6), 
                                                                     int(duration * 1e3), wait_after_trigger, int(time.time()))
    with open(file_name, 'w') as f:
        f.write("round,ac_source,point_count,average_v, std_v, dc_current\n")

    for i in range(round):
        print("round: " + str(i+1))
        for current in ['ac_1', 'ac_2']:
            print("current:" + current)
            for point in range(point_per_round):
                if current == 'ac_1':
                    trigger_ac(ac_1, offs=ac_current, duration=duration)

                else:
                    trigger_ac(ac_1, offs=-1 * ac_current, duration=duration)
                time.sleep(wait_after_trigger)
                average_v, std_v = measure_resistance_with_off_set(ac=ac_1, volt=volt, measure_points=30,dc_current=100e-6, duration=4)
                with open(file_name, 'a') as f:
                    f.write("{},{},{},{},{},{}\n".format(i, current, point, average_v, std_v, dc_current))
                print(average_v)
                time.sleep(20)
                """
                plt.clf()
                df = pd.read_csv(file_name)
                v = df['average_v']
                v.plot()
                plt.show()"""

In [12]:
import subprocess
def set_T(t=300):
    if t < 2 or t > 400:
        raise
    result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms.py', str(t)])

    current_t = result.decode('utf-8').strip()
    if abs(float(current_t)-t)<1:
        print("current T {}".format(current_t))
        return
    else:
        print("set t:")
        print(t)
        print("current t:")
        print(current_t)
        raise

In [53]:
result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_set_position.py', '0'])

In [10]:

def set_Field(field=0, persistent=False):
    if abs(field) > 140000:
        raise
    if persistent:
        result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_set_field.py',
                                          str(field), str('persistent')])
    else:
        result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                          r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_set_field.py', 
                                          str(field)])

    current_field = result.decode('utf-8').strip()
    print(current_field)
    if abs(float(current_field)-field)<2:
        print("current field {}".format(current_field))
        return
    else:
        print("set field:")
        print(field)
        print("current t:")
        print(current_field)
        raise

In [11]:
def set_position(p):
    if p >360 and p < -10:
        raise
    subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_move_p.py',
                                          str(p)])
    

In [14]:
set_position(1)

In [20]:
set_Field(10)

9.95123958588
current field 9.95123958588


In [20]:
set_T(300)

current T 299.9991


In [142]:
do_task = True

In [228]:
#normal two leg switch
#set_Field(0)
#time.sleep(60)
if not do_task:
    raise
#t_array = [100, 200, 250, 270, 280, 290, 300, 310, 320,330, 340, 350]
t_array = [300]
#i = [8*1e-3]

i = [9*1e-3]
fields = [0]
wait_after_T = 15 # in minute
for t in t_array:
    print(datetime.datetime.now())
    print("set T to:" + str(t))
    print(datetime.datetime.now())
    print("start measurment:")
    #set_T(t)
    #time.sleep(wait_after_T*60)
    for c in i:
        for field in fields:
            print("current pulse:" + str(c))
            print("set field to:" + str(field))
            #set_Field(field)
            print(datetime.datetime.now())
            do_measurement(sample_id='XD281_1_nm_pt_30nm_Fe2O3_0001__90degree_cross_pulse_8legs_10um_DC_high_impedance_current_dependent'+ str(t) +"K" + str(field) + "G",
                           ac_current=c, dc_current=100e-6, duration=0.001, 
                           round=4, point_per_round=5, wait_after_trigger=30)
do_task = False

2018-12-27 18:04:48.729000
set T to:300
2018-12-27 18:04:48.737000
start measurment:
current pulse:0.014
set field to:0
2018-12-27 18:04:48.737000
round: 1
current:ac_1
1.6511300839285714e-06
1.3204936182142858e-06
1.22723055375e-06
8.316197561071428e-07
3.6790154579285714e-07
current:ac_2
1.0530901691071428e-06
1.012048133392857e-06
1.4389538676428573e-06
1.838966205357143e-06
1.8382588046428572e-06
round: 2
current:ac_1
1.5453319825000002e-06
1.5418529929999999e-06
1.3798818449285714e-06
1.200634152892857e-06
1.122353120357143e-06
current:ac_2
1.6091227485714287e-06
1.5760265032142857e-06
2.044746595e-06
2.058869769285714e-06
2.37903402e-06
round: 3
current:ac_1


PermissionError: [Errno 13] Permission denied: './_switch/data/XD281_1_nm_pt_30nm_Fe2O3_0001__90degree_cross_pulse_8legs_10um_DC_high_impedance_current_dependent300K0Gac_14000uA1ms_wait30s_1545951888.csv'

In [27]:
set_Field(10)

CalledProcessError: Command '['C:\\Program Files\\IronPython 2.7\\ipy.exe', 'C:\\Users\\yanglabuser\\Desktop\\Sam files\\_switch\\ppms-master\\ppms_set_field.py', '10']' returned non-zero exit status 1.

In [36]:
do_task = True
check_conn = False
anneal_at_high_T = True

In [37]:
#three leg switch, up mid down mid up
#set_Field(0)
#time.sleep(60)

if not do_task:
    raise
#t_array = [100, 200, 250, 270, 280, 290, 300, 310, 320,330, 340, 350]
#t_array = [100, 150, 200, 225, 250, 275, 300, 325]
t_array = [300]
#i = [15*1e-3]
#     14*1e-3, 16*1e-3,16*1e-3, 14*1e-3, 12*1e-3,10*1e-3, 8*1e-3, 6*1e-3]
#i = [15*1e-3]#[::-1]


i = [8*1e-3, 10*1e-3,12*1e-3,14*1e-3,16*1e-3,17*1e-3]

i = i + i[::-1]

#set_T(300)
#time.sleep(15*60)
#i = [10*1e-3]
#set_Field(30000, True)

pulse_durations = [0.001]
fields = [0]
wait_after_T = 5 # in minute
if check_conn:
    do_measurement_three_pulses(sample_id='D807_5nm_MnSn__8leg_0dsegree_0T_switch_10sec_3sec__345Kanneal'+ "G",
                   ac_current=10*1e-3, dc_current=10e-5, duration=0.001, 
                   round=2, point_per_round=4, wait_after_trigger=10, pulse=['ac_1', 'ac_2'])
    raise

if anneal_at_high_T:
    set_T(350)
    do_measurement_three_pulses(sample_id='D807_5nm_MnSn__8leg_0dsegree_0T_switch_10sec_3sec__345Kanneal'+ "G",
               ac_current=17*1e-3, dc_current=10e-5, duration=0.001, 
               round=4, point_per_round=10, wait_after_trigger=10, pulse=['ac_1', 'ac_2'])

for t in t_array:
    print(datetime.datetime.now())
    print("set T to:" + str(t))
    print(datetime.datetime.now())
    print("start measurment:")
    if not t is None:
        set_T(t)
        time.sleep(wait_after_T*60)
    
    for c in i:
        for field in fields:
            for pulse_d in pulse_durations:
                print("current pulse:" + str(c))
                print("set field to:" + str(field))
                #set_Field(field)
                print(datetime.datetime.now())
                do_measurement_three_pulses(sample_id='D807_5nm_MnSn__8leg_0dsegree_0T_switch_10sec_3sec_after345Kanneal'+ str(t) +"K" + str(field) + "G",
                               ac_current=c, dc_current=10e-5, duration=pulse_d, 
                               round=4, point_per_round=10, wait_after_trigger=10, pulse=['ac_1', 'ac_2'])

do_task = False

current T 350.0796
round: 1
current:ac_1
-2.2133362064285715e-05
-2.2424985882142858e-05
-2.1225609082142857e-05
-1.9220203507142855e-05
-1.7492346264285716e-05
-1.644023194642857e-05
-1.5166941232142853e-05
-1.5192682042857142e-05
-1.4927965746428572e-05
-1.4447674846428568e-05
current:ac_2
-1.5151399492857145e-05
-1.6418902121428568e-05
-1.808079895e-05
-1.812605963214285e-05
-1.839336891428571e-05
-1.8849849985714285e-05
-1.9435903667857144e-05
-2.020430351785714e-05
-2.024712637142857e-05
-2.0483109353571425e-05
round: 2
current:ac_1
-1.9897912346428572e-05
-1.9917848910714286e-05
-1.9780502564285716e-05
-1.9518112524999997e-05
-1.907449067142857e-05
-1.8650765053571424e-05
-1.813566792857143e-05
-1.7541322164285714e-05
-1.748335203214286e-05
-1.7448947260714285e-05
current:ac_2
-1.8014693321428576e-05
-1.8769829639285713e-05
-1.9419641714285712e-05
-1.9432827385714286e-05
-1.9482094135714287e-05
-1.9578578335714288e-05
-1.9837668328571426e-05
-2.0184756457142855e-05
-2.06164065750

-2.0635819828571426e-05
-2.0637072571428568e-05
-2.0636411632142858e-05
-2.063036429285714e-05
-2.062114525357143e-05
current pulse:0.014
set field to:0
2020-03-11 15:46:48.360307
round: 1
current:ac_1
-2.064915414285714e-05
-2.0646218825000002e-05
-2.065019727857143e-05
-2.066224945714286e-05
-2.0675340232142862e-05
-2.0673365110714283e-05
-2.0676842532142862e-05
-2.0664726689285712e-05
-2.0667909567857146e-05
-2.0672067825000002e-05
current:ac_2
-2.0652883760714286e-05
-2.0644200210714287e-05
-2.0632566160714286e-05
-2.0628557689285712e-05
-2.0632532878571423e-05
-2.0637784253571426e-05
-2.0626436992857138e-05
-2.0631520432142856e-05
-2.0610755360714282e-05
-2.0624656085714285e-05
round: 2
current:ac_1
-2.0628365925000004e-05
-2.0638729264285717e-05
-2.0639012410714285e-05
-2.0641069075e-05
-2.0642183035714288e-05
-2.064068958214285e-05
-2.06474164e-05
-2.0645575721428575e-05
-2.0647907571428574e-05
-2.065461492142857e-05
current:ac_2
-2.0633328178571426e-05
-2.0632321567857142e-05
-

-2.0612991342857144e-05
-2.0584299635714283e-05
-2.0552603100000004e-05
-2.0541914542857143e-05
-2.0535160571428576e-05
-2.0536783671428573e-05
-2.0527728014285712e-05
current pulse:0.016
set field to:0
2020-03-11 17:46:10.293660
round: 1
current:ac_1
-2.0560795274999998e-05
-2.0579225446428575e-05
-2.0590611589285713e-05
-2.060217845357143e-05
-2.061123825e-05
-2.0610627021428567e-05
-2.0615463778571426e-05
-2.060704166428571e-05
-2.062403271785714e-05
-2.0630603800000003e-05
current:ac_2
-2.061354381071428e-05
-2.060030268214285e-05
-2.059725440714286e-05
-2.0588576185714287e-05
-2.0573965099999996e-05
-2.056633535357143e-05
-2.056369255e-05
-2.0541279085714288e-05
-2.052423382857143e-05
-2.050830617142857e-05
round: 2
current:ac_1
-2.0535637210714287e-05
-2.0550800289285712e-05
-2.0559518610714287e-05
-2.0578846346428574e-05
-2.0595494717857143e-05
-2.0582183225e-05
-2.059147487857143e-05
-2.0591738382142857e-05
-2.0603612875000003e-05
-2.0598374932142856e-05
current:ac_2
-2.0588130

-2.0617921857142857e-05
-2.061928216428571e-05
-2.061310960357143e-05
-2.0619293978571426e-05
-2.0620870239285712e-05
-2.062286320714286e-05
-2.0622089739285718e-05
-2.061406254642857e-05
-2.0619202900000004e-05
current pulse:0.008
set field to:0
2020-03-11 19:45:32.642895
round: 1
current:ac_1
-2.0616578321428575e-05
-2.0617730507142857e-05
-2.061300419285715e-05
-2.0611149617857142e-05
-2.0620642924999998e-05
-2.0615037957142856e-05
-2.0625334728571426e-05
-2.0622987717857143e-05
-2.0613309217857144e-05
-2.0626562525000002e-05
current:ac_2
-2.0617496642857142e-05
-2.062399687142857e-05
-2.0626431949999998e-05
-2.0631513749999996e-05
-2.0617224460714287e-05
-2.0625444303571426e-05
-2.061878917142857e-05
-2.0624855989285717e-05
-2.062040535357143e-05
-2.0618052317857144e-05
round: 2
current:ac_1
-2.061721795357143e-05
-2.0615086832142854e-05
-2.0624092360714282e-05
-2.0625436514285716e-05
-2.0623309239285717e-05
-2.063452889642858e-05
-2.0622801064285712e-05
-2.062585855714286e-05
-2.0

In [64]:
def angular_measurement(dc, volt, sample_id="Fe2O3_alpha_hall_scan", dc_current=0.0001,  
                        time_per_point=2, time_out_count = 30):
    filename = "./_switch/data/" + sample_id + "_{}.csv".format(int(time.time()))
    with open(filename, 'w') as f:
        f.write("average_v, std_v, dc_current, POS\n")
    point_count = 0
    count = 0 # if position didn't change count++, stop measurement once count > 
    last_position = None
    while count < time_out_count:
        pos = get_position()
        print("current position: {}, count:{}".format(pos, count))
        if pos == last_position and count == 1:
            count += 1
            average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
            with open(filename, 'a') as f:
                f.write("{},{},{},{}\n".format(average_v, std_v, dc_current, pos))
        elif pos == last_position:
            count += 1
        else:
            count =0
            last_position = pos
        time.sleep(time_per_point)
                

In [51]:
time.sleep(1.5)

In [66]:
if do_task:
    angular_measurement(dc, volt, sample_id="Xd357_2_Fe2O330nm_3nmW__8leg_0dsegree_Hall_alpha_1T300K", dc_current=0.0001,  
                        time_per_point=1.5, time_out_count=30)
    do_task=False
else:
    raise 
    

current position: 0, count:0
current position: 0, count:0
current position: 0, count:1
current position: 0, count:2
current position: 0, count:3
current position: 0, count:4
current position: 0, count:5
current position: 5, count:6
current position: 5, count:0
current position: 5, count:1
current position: 5, count:2
current position: 5, count:3
current position: 5, count:4
current position: 6, count:5
current position: 10, count:0
current position: 10, count:0
current position: 10, count:1
current position: 10, count:2
current position: 10, count:3
current position: 10, count:4
current position: 15, count:5
current position: 15, count:0
current position: 15, count:1
current position: 15, count:2
current position: 15, count:3
current position: 15, count:4
current position: 15, count:5
current position: 20, count:6
current position: 20, count:0
current position: 20, count:1
current position: 20, count:2
current position: 20, count:3
current position: 20, count:4
current position: 23, co

current position: 205, count:0
current position: 205, count:1
current position: 205, count:2
current position: 205, count:3
current position: 205, count:4
current position: 210, count:5
current position: 210, count:0
current position: 210, count:1
current position: 210, count:2
current position: 210, count:3
current position: 210, count:4
current position: 213, count:5
current position: 215, count:0
current position: 215, count:0
current position: 215, count:1
current position: 215, count:2
current position: 215, count:3
current position: 215, count:4
current position: 220, count:5
current position: 220, count:0
current position: 220, count:1
current position: 220, count:2
current position: 220, count:3
current position: 220, count:4
current position: 225, count:5
current position: 225, count:0
current position: 225, count:1
current position: 225, count:2
current position: 225, count:3
current position: 225, count:4
current position: 228, count:5
current position: 230, count:0
current 

In [95]:
#using offset
#set_Field(0)
#time.sleep(60)
if not do_task:
    raise
#t_array = [100, 200, 250, 270, 280, 290, 300, 310, 320,330, 340, 350]
t_array = [300]
i = [11*1e-3]
fields = [1000]
wait_after_T = 15 # in minute
for t in t_array:
    print(datetime.datetime.now())
    print("set T to:" + str(t))
    print(datetime.datetime.now())
    print("start measurment:")
    #set_T(t)
    #time.sleep(wait_after_T*60)
    for c in i:
        for field in fields:
            print("current pulse:" + str(c))
            print("set field to:" + str(field))
            #set_Field(field)
            print(datetime.datetime.now())
            do_measurement_with_off_set(sample_id='XD330_2_3nm_Cr2O3_0001_pt_current_pulse_dc__direction_field_dc'+ str(t) +"K" + str(field) + "G",
                           ac_current=c, dc_current=100e-6, duration=0.001, 
                           round=4, point_per_round=20, wait_after_trigger=30)
do_task = False

2018-12-19 15:00:26.966000
set T to:300
2018-12-19 15:00:26.967000
start measurment:
current pulse:0.011
set field to:1000
2018-12-19 15:00:26.967000
round: 1
current:ac_1
-5.199585600714286e-05
-5.189326961071429e-05
-5.205547158214287e-05
-5.189641496785715e-05
-5.1944225771428573e-05
-5.199308106428571e-05
-5.195535313214286e-05
-5.191972721785715e-05
-5.1974770171428586e-05
-5.201323540714287e-05
-5.196443363214287e-05
-5.195315543928571e-05
-5.1888259821428575e-05
-5.1996152385714285e-05
-5.197224703571428e-05
-5.1864810014285725e-05
-5.193070986785715e-05
-5.193894402500001e-05
-5.195810858928571e-05
-5.1874144825e-05
current:ac_2
-5.194893401428571e-05
-5.187097353214286e-05
-5.189259728571427e-05
-5.1946196542857144e-05
-5.1930558849999994e-05


PermissionError: [Errno 13] Permission denied: './_switch/data/XD330_2_3nm_Cr2O3_0001_pt_current_pulse_dc__direction_field_dc300K1000Gac_11000uA1ms_wait30s_1545249626.csv'

In [81]:
def field_cool(field=0, upper_T=360, lower_T=150, time_at_upper_t=10, time_at_lower_t=5, back_to_t=300,
               back_to_field=0, pulse_round=0, c=11*1e-3, sample_id=""):
    print("set T " + str(upper_T))
    set_T(upper_T)
    time.sleep(time_at_upper_t * 60)
    if pulse_round > 0:
        do_measurement(sample_id= sample_id + "pulses_at_360K", ac_current=c, dc_current=100e-6, duration=0.001,
                       round=pulse_round, point_per_round=10, wait_after_trigger=30)
    print("set field " + str(field))
    set_Field(field)
    print("set T " + str(lower_T))
    set_T(lower_T)
    time.sleep(time_at_lower_t * 60)
    print("set field " + str(back_to_field))
    set_Field(back_to_field)
    print("set T " + str(back_to_t))
    set_T(back_to_t)
    print("field cool finished")


In [82]:
# field cool
if not do_task:
    raise

t_array = [300]
i = [11*1e-3]
fields = [140000]
t_wait = 15
for t in t_array:
    for c in i:
        for field in fields:
            print("start current field cool with in plane field:" + str(field))
            print(datetime.datetime.now())
            field_cool(field=field, upper_T=360, lower_T=150, time_at_upper_t=1, time_at_lower_t=1, back_to_t=300,
               back_to_field=0, pulse_round=4, c=11*1e-3, sample_id="XD330_1")
            time.sleep(t_wait * 60)
            do_measurement(sample_id='XD330_1_2_11_22_3nm_Cr2O3_0001_pt_'+ str(t) +"K_in_plan_field_cool_" + str(field) + "G",
                           ac_current=c, dc_current=100e-6, duration=0.001, 
                           round=4, point_per_round=10, wait_after_trigger=30)
do_task = False
    

start current field cool with field:140000
2018-12-09 02:24:10.526000
set T 360
current T 360.082
round: 1
current:ac_1
-8.777724527142856e-05
-8.782709108214284e-05
-8.762150914999999e-05
-8.724146636428572e-05
-8.68492061107143e-05
-8.647629828571429e-05
-8.602266202142856e-05
-8.551312789642856e-05
-8.517479112142858e-05
-8.495208869285715e-05
current:ac_2
-8.001586415e-05
-8.038368485714287e-05
-8.079789659285715e-05
-8.009619972142856e-05
-8.04840977e-05
-8.073744913214286e-05
-8.039684221428572e-05
-8.03820156392857e-05
-8.059263045714285e-05
-8.049004140357142e-05
round: 2
current:ac_1
-8.193233196428572e-05
-8.320079210357144e-05
-8.381387909642858e-05
-8.449658576428572e-05
-8.465868791428572e-05
-8.448224749285716e-05
-8.437397411071429e-05
-8.421407610357142e-05
-8.403173252142857e-05
-8.453799031785716e-05
current:ac_2
-8.100591894285715e-05
-8.071556498571428e-05
-8.093236766428571e-05
-8.103985727142858e-05
-8.143930406428573e-05
-8.106861351071428e-05
-8.080103772499999e

-7.948244577857143e-05
-7.948694011428571e-05
-7.954703552857141e-05


PermissionError: [Errno 13] Permission denied: './_switch/data/XD330_1_2_11_22_3nm_Cr2O3_0001_pt_300K_out_plan_field_cool_20000Gac_11000uA1ms_wait30s_1544370619.csv'

In [95]:
# switch with field
if not do_task:
    raise

t_array = [300, 350]
i = [11*1e-3]
fields = [140000]
t_wait = 10
set_Field(fields[0])
for t in t_array:
    print("set T:" + str(t))
    set_T(t)
    for c in i:
        for field in fields:
            print("start measurement with field:" + str(field))
            print(datetime.datetime.now())
            if t == 350:
                time.sleep(t_wait * 60)
            do_measurement(sample_id='XD330_1_2_11_22_3nm_Cr2O3_0001_pt_'+ str(t) +"K_switch_in_plan_45_field_" + str(field) + "G",
                           ac_current=c, dc_current=100e-6, duration=0.001, 
                           round=4, point_per_round=10, wait_after_trigger=30)
do_task = False
    

140000.5
current field 140000.5
set T:300
current T 300.0091
start measurement with field:140000
2018-12-09 23:14:00.835000
round: 1
current:ac_1
7.754044281428572e-05
7.749976215357143e-05
7.754495433571428e-05
7.755004506071429e-05
7.750008960000001e-05
7.749854076428571e-05
7.750188894642858e-05
7.746885484642857e-05
7.745733136785715e-05
7.747724496071429e-05
current:ac_2
7.753101659285715e-05
7.762534457499999e-05
7.767738074642856e-05
7.766846882857143e-05
7.769023476428571e-05
7.771267387500001e-05
7.7736994475e-05
7.774105442857143e-05
7.773479446428571e-05
7.7743676125e-05
round: 2
current:ac_1
7.776682756428571e-05
7.764882875357142e-05
7.7519248875e-05
7.753216649999999e-05
7.74493368e-05
7.735644509285715e-05
7.726701748928571e-05
7.719531448214286e-05
7.725777393214286e-05
7.714064761071428e-05
current:ac_2
7.720172053571429e-05
7.72302894e-05
7.731252555357143e-05
7.738817081428572e-05
7.739694820714285e-05
7.740350225357143e-05
7.739239584642857e-05
7.739768606785714e-05

PermissionError: [Errno 13] Permission denied: './_switch/data/XD330_1_2_11_22_3nm_Cr2O3_0001_pt_300K_switch_in_plan_45_field_140000Gac_11000uA1ms_wait30s_1544415240.csv'

In [98]:
# switch with field
if not do_task:
    raise

t_array = [350]
i = [11*1e-3]
fields = [140000]
t_wait = 10
#set_Field(fields[0])
for t in t_array:
    print("set T:" + str(t))
    #set_T(t)
    for c in i:
        for field in fields:
            print("start measurement with field:" + str(field))
            print(datetime.datetime.now())
            #if t == 350:
            #    time.sleep(t_wait * 60)
            do_measurement(sample_id='XD330_1_2_11_22_3nm_Cr2O3_0001_pt_'+ str(t) +"K_switch_in_plan_45_field_" + str(field) + "G",
                           ac_current=c, dc_current=100e-6, duration=0.001, 
                           round=3, point_per_round=10, wait_after_trigger=30)


print("45 degree finished")
print("rotate to 90 degree")
time.sleep(30*60)
# field cool
t_array = [300]
i = [11*1e-3]
fields = [140000]
t_wait = 15
for t in t_array:
    for c in i:
        for field in fields:
            print("start current field cool with 90 in plane field:" + str(field))
            print(datetime.datetime.now())
            field_cool(field=field, upper_T=350, lower_T=150, time_at_upper_t=1, time_at_lower_t=1, back_to_t=300,
               back_to_field=0, pulse_round=0, c=11*1e-3, sample_id="XD330_1")
            time.sleep(t_wait * 60)
            do_measurement(sample_id='XD330_1_2_11_22_3nm_Cr2O3_0001_pt_'+ str(t) +"K_in_plan_field_90_cool_" + str(field) + "G",
                           ac_current=c, dc_current=100e-6, duration=0.001, 
                           round=4, point_per_round=10, wait_after_trigger=30)
            
# switch with field

t_array = [350, 300]
i = [11*1e-3]
fields = [140000]
t_wait = 15
set_Field(fields[0])
for t in t_array:
    print("set T:" + str(t))
    set_T(t)
    for c in i:
        for field in fields:
            print("start measurement with field:" + str(field))
            print(datetime.datetime.now())
            time.sleep(t_wait * 60)
            do_measurement(sample_id='XD330_1_2_11_22_3nm_Cr2O3_0001_pt_'+ str(t) +"K_switch_with_90_in_plane__plan_field_" + str(field) + "G",
                           ac_current=c, dc_current=100e-6, duration=0.001, 
                           round=4, point_per_round=10, wait_after_trigger=30)
do_task = False
    

RuntimeError: No active exception to reraise

In [99]:
set_Field(0)

-0.026669986546
current field -0.026669986546


In [16]:
def stable_measurement(sample_id='', ac_current=5*1e-3, dc_current=100e-6, duration=0.005,
                       round=6, point_per_round=1, wait_after_trigger=15, dc_measure_delay=60, dc_measurement_count=30):
    file_name = "./_switch/data/{}ac_{}uA{}ms_wait{}s.csv".format(sample_id, int(ac_current * 1e6), int(duration * 1e3), wait_after_trigger)
    with open(file_name, 'w') as f:
        f.write("round,ac_source,point_count,average_v, std_v, dc_current\n")

    for i in range(round):
        print("round: " + str(i+1))
        for current in ['ac_1', 'ac_2']:
            print("current:" + current)
            for point in range(point_per_round):
                if current == 'ac_1':
                    trigger_ac(ac_1, offs=ac_current, duration=duration)

                else:
                    trigger_ac(ac_2, offs=ac_current, duration=duration)
                time.sleep(wait_after_trigger)
                for i in range(dc_measurement_count):
                    time.sleep(dc_measure_delay)
                    average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
                    with open(file_name, 'a') as f:
                        f.write("{},{},{},{},{},{}\n".format(i, current, point, average_v, std_v, dc_current))
                    print(average_v)
                time.sleep(20)

In [18]:
stable_measurement(sample_id='30nm_CrO_0001_330K_stable_1min_delay_30_count', ac_current=c, dc_current=100e-6, duration=0.001,
                   round=10, point_per_round=1, wait_after_trigger=30, dc_measure_delay=60, dc_measurement_count=30)

round: 1
current:ac_1
9.431330707142858e-05
9.43381000142857e-05
9.433512416428573e-05
9.434283410357144e-05
9.433206410714284e-05
9.433824302142855e-05
9.43364345857143e-05
9.431444373928572e-05
9.432693993214288e-05
9.431336129285715e-05
9.430465472857142e-05
9.430196330714288e-05
9.429996161785713e-05
9.430293407857143e-05
9.43006512107143e-05
9.428696045714285e-05
9.430295150357143e-05
9.428607778214287e-05
9.428342891785712e-05
9.429738762142855e-05
9.429405230357143e-05
9.427626310357143e-05
9.427893440714288e-05
9.426475354285716e-05
9.426020255714289e-05
9.426062128571426e-05
9.426518625357144e-05
9.427324081071429e-05
9.426125397857145e-05
9.424162587142856e-05
current:ac_2
9.49565684857143e-05
9.493103777499998e-05
9.491215283571429e-05
9.487708946785713e-05
9.488089966428574e-05
9.48385205714286e-05
9.483650877500002e-05
9.484322112857143e-05
9.482787673928571e-05
9.481444497142855e-05
9.482042050714288e-05
9.483155735714285e-05
9.4829030425e-05
9.482499039642856e-05
9.48133

PermissionError: [Errno 13] Permission denied: './_switch/data/30nm_CrO_0001_330K_stable_1min_delay_30_countac_10000uA1ms_wait30s.csv'

In [6]:
inst = rm.open_resource('GPIB0::7::INSTR')

In [7]:
inst.write("initiate")

(10, <StatusCode.success: 0>)

In [14]:
keithley = Keithley2400("GPIB::{}".format(dc_2400_id))
inst = rm.open_resource('GPIB0::{}::INSTR'.format(nanovolt_id))

In [42]:
keithley.apply_current()# Sets up to source current
keithley.source_current_range = 10e-3   # Sets the source current range to 10 mA
keithley.compliance_voltage = 10        # Sets the compliance voltage to 10 V
keithley.source_current = 100e-6            # Sets the source current to 0 mA
keithley.enable_source() 
v = []
average_v = []
for i in range(30):
    if i < 10:
        continue
    time.sleep(0.1)
    v.append(abs(float(inst.query("fetch?"))))

# Sets the compliance voltage to 10 V
keithley.source_current = -100e-6            # Sets the source current to 0 mA
keithley.enable_source() 

for i in range(30):
    time.sleep(0.1)
    if i < 10:
        continue
    v.append(abs(float(inst.query("fetch?"))))
v = sorted(v)
v = v[5: -5]

with open("/Users/sishengyu/Desktop/abc_3.csv", 'a') as f:
    print(np.array(v).mean())
    f.write(str(np.array(v).mean()))
    f.write(",")
    f.write(str(np.array(v).std()))
    f.write(",")
    f.write(str(np.array(v).std()/np.array(v).mean()))
    f.write("\n")

0.0003667652223333333


In [26]:
fs = [1000]
steps = [100]

In [17]:
def generate_fields(fileds, steps):
    assert (len(fileds) == len(steps))
    res = []
    for i in range(len(fileds)):
        res.extend([f for f in range(fileds[i], fileds[i + 1] if i+1 < len(fileds) else 0, 
                                   -1 * steps[i])])
    res = res + [0] + [-1* f for f in res[::-1]]
    res += res[::-1][1:]
    return res  

In [18]:
fields = generate_fields(fs, steps)
print(fields)

NameError: name 'fs' is not defined

In [17]:
fields = list(range(0, 140000, 5000)) + list(range(140000,0,-5000)) + [0]
print(fields)

[0, 5000, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000, 55000, 60000, 65000, 70000, 75000, 80000, 85000, 90000, 95000, 100000, 105000, 110000, 115000, 120000, 125000, 130000, 135000, 140000, 135000, 130000, 125000, 120000, 115000, 110000, 105000, 100000, 95000, 90000, 85000, 80000, 75000, 70000, 65000, 60000, 55000, 50000, 45000, 40000, 35000, 30000, 25000, 20000, 15000, 10000, 5000, 0]


In [29]:
def field_scan(dc, volt, dc_current, fields, Ts=[None], sample_id="", t_wait=10, field_wait=5):
    for T in Ts:
        if not T is None:
            set_T(T)
            time.sleep(t_wait*60)
        else:
            T = "300"
            
        file_name = "./_switch/data/" + sample_id + str(T) + "K" + str(int(time.time())) + ".csv"
        with open(file_name, 'w') as f:
            f.write("field(G), average_v, std_v, dc_current\n")
            
        for field in fields:
            set_Field(field)
            time.sleep(field_wait)
            average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
            with open(file_name, 'a') as f:
                f.write("{},{},{},{}\n".format(field, average_v, std_v, dc_current))

In [19]:
def field_scan_two_sample(dc1, volt1, dc2, volt2, dc_current_1, dc_current_2,fields=[], Ts=[None], sample1_id="1", sample2_id="2", t_wait=10, field_wait=5):
    for T in Ts:
        if not T is None:
            set_T(T)
            print(datetime.datetime.now())
            time.sleep(t_wait*60)
        else:
            T = "300"
            
        file_1 = "./_switch/data/" + sample1_id + "_" + str(T) + "K_" + str(int(1e6*dc_current_1))+ "uA" + str(int(time.time())) + ".csv"
        file_2 = "./_switch/data/" + sample2_id + "_" + str(T) + "K_" + str(int(1e6*dc_current_2))+ "uA" + str(int(time.time())) + ".csv"
        
        with open(file_1, 'w') as f:
            f.write("field(G), average_v, std_v, dc_current\n")
            
        with open(file_2, 'w') as f:
            f.write("field(G), average_v, std_v, dc_current\n")
            
        for field in fields:
            set_Field(field)
            time.sleep(field_wait)
            for file in [file_1, file_2]:
                if file == file_1:
                    dc_current = dc_current_1
                    average_v, std_v = measure_resistance(dc=dc1, volt=volt1, dc_current=dc_current,measure_points=50, skip_points=20)
                else:
                    dc_current = dc_current_2
                    average_v, std_v = measure_resistance(dc=dc2, volt=volt2, dc_current=dc_current,measure_points=50, skip_points=20)
                with open(file, 'a') as f:
                    f.write("{},{},{},{}\n".format(field, average_v, std_v, dc_current))

In [19]:
def field_scan_lock_in(lock, first_range=0.1, second_range=0.00001, time_step=0.1, input_I =1, wait_before_measure=10, 
                       count=10, fields=[], Ts=[None], sample1_id="1", t_wait=10, field_wait=5):
    for T in Ts:
        if not T is None:
            set_T(T)
            print(datetime.datetime.now())
            time.sleep(t_wait*60)
        else:
            T = "300"
            
        file_1 = "./_switch/data/" + sample1_id + "_" + str(T) + "K_" + str(input_I) + "mA" + str(int(time.time())) + ".csv"
        
        with open(file_1, 'w') as f:
            f.write("field(G),x,y,r,theta,std_x,std_y,std_r,std_theta,"
                    "x2,y2,r2,theta2,std_x2,std_y2,std_r2,std_theta2,"
                    "input_I\n")
            
        for field in fields:
            set_Field(field)
            time.sleep(field_wait)
            print("phase 1")
            res1 = lock.lock_in_measure(count, time_step, wait_before_measure, harm=1,sens=first_range)
            print(res1)
            print("phase2")
            res2 = lock.lock_in_measure(count, time_step, wait_before_measure, harm=2,sens=second_range)
            print(res2)

            with open(file_1, 'a') as f:
                f.write("{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(field, *res1, *res2 ,input_I*0.001))

In [21]:
def field_scan_two_sample_two_channel(dc1, volt1, dc2, volt2, dc_current_1=1e-4, dc_current_2=1e-4, fields=[], Ts=[None], sample1_id="1", sample2_id="2",
                                      t_wait=10, field_wait=5):
    for T in Ts:
        if not T is None:
            set_T(T)
            print(datetime.datetime.now())
            time.sleep(t_wait*60)
        else:
            T = "300"
            
        file_1 = "./_switch/data/" + sample1_id + "_" + str(T) + "K_" + str(int(dc_current_1*1e6)) + "uA"+ str(int(time.time())) + ".csv"
        file_2 = "./_switch/data/" + sample2_id + "_" + str(T) + "K_" + str(int(dc_current_2*1e6)) + "uA"+ str(int(time.time())) + ".csv"
        
        with open(file_1, 'w') as f:
            f.write("field(G), average_v_1, std_v_1, average_v_2, std_v_2, dc_current\n")
            
        with open(file_2, 'w') as f:
            f.write("field(G), average_v_1, std_v_1,  average_v_2, std_v_2, dc_current\n")
        dc_current = None
        for field in fields:
            set_Field(field)
            time.sleep(field_wait)
            for file in [file_1, file_2]:
                if file == file_1:
                    dc_current = dc_current_1
                    average_v_1, std_v_1, average_v_2, std_v_2 = measure_resistance_two_channel(dc=dc1, volt=volt1, dc_current=dc_current)
                else:
                    dc_current = dc_current_2
                    average_v_1, std_v_1, average_v_2, std_v_2 = measure_resistance_two_channel(dc=dc2, volt=volt2, dc_current=dc_current)
                    
                with open(file, 'a') as f:
                    f.write("{},{},{},{},{},{}\n".format(field, average_v_1, std_v_1, average_v_2, std_v_2, dc_current))

In [22]:
do_task = True

In [406]:
#fields_start = list(range(0, 50000, 10000)) #+ list(range(-300, -1000, -100)) + list(range(-1000,-10000, -1000))

In [158]:
set_Field(0, True)

-0.000416718539782
current field -0.000416718539782


In [87]:
fields

[60000,
 55000,
 50000,
 48000,
 46000,
 44000,
 42000,
 40000,
 38000,
 36000,
 34000,
 32000,
 30000,
 28000,
 26000,
 24000,
 22000,
 20000,
 18000,
 16000,
 14000,
 12000,
 10000,
 8000,
 6000,
 4000,
 2000,
 0,
 -2000,
 -4000,
 -6000,
 -8000,
 -10000,
 -12000,
 -14000,
 -16000,
 -18000,
 -20000,
 -22000,
 -24000,
 -26000,
 -28000,
 -30000,
 -32000,
 -34000,
 -36000,
 -38000,
 -40000,
 -42000,
 -44000,
 -46000,
 -48000,
 -50000,
 -55000,
 -60000,
 -55000,
 -50000,
 -48000,
 -46000,
 -44000,
 -42000,
 -40000,
 -38000,
 -36000,
 -34000,
 -32000,
 -30000,
 -28000,
 -26000,
 -24000,
 -22000,
 -20000,
 -18000,
 -16000,
 -14000,
 -12000,
 -10000,
 -8000,
 -6000,
 -4000,
 -2000,
 0,
 2000,
 4000,
 6000,
 8000,
 10000,
 12000,
 14000,
 16000,
 18000,
 20000,
 22000,
 24000,
 26000,
 28000,
 30000,
 32000,
 34000,
 36000,
 38000,
 40000,
 42000,
 44000,
 46000,
 48000,
 50000,
 55000,
 60000]

In [29]:
fs = [60000, 50000]
steps = [5000, 2000]
fields = generate_fields(fs, steps)
print(fields)

[60000, 55000, 50000, 48000, 46000, 44000, 42000, 40000, 38000, 36000, 34000, 32000, 30000, 28000, 26000, 24000, 22000, 20000, 18000, 16000, 14000, 12000, 10000, 8000, 6000, 4000, 2000, 0, -2000, -4000, -6000, -8000, -10000, -12000, -14000, -16000, -18000, -20000, -22000, -24000, -26000, -28000, -30000, -32000, -34000, -36000, -38000, -40000, -42000, -44000, -46000, -48000, -50000, -55000, -60000, -55000, -50000, -48000, -46000, -44000, -42000, -40000, -38000, -36000, -34000, -32000, -30000, -28000, -26000, -24000, -22000, -20000, -18000, -16000, -14000, -12000, -10000, -8000, -6000, -4000, -2000, 0, 2000, 4000, 6000, 8000, 10000, 12000, 14000, 16000, 18000, 20000, 22000, 24000, 26000, 28000, 30000, 32000, 34000, 36000, 38000, 40000, 42000, 44000, 46000, 48000, 50000, 55000, 60000]


In [35]:
fields = list(range(0, 3000, 300)) + list(range(3000, 10000, 1000)) + list(range(10000, 3000, -1000)) +  list(range(3000, -300, -300))
print(fields)

[0, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 9000, 8000, 7000, 6000, 5000, 4000, 3000, 2700, 2400, 2100, 1800, 1500, 1200, 900, 600, 300, 0]


In [87]:
set_Field(29990)

29990.0839844
current field 29990.0839844


In [23]:
fs = [140000]
steps = [5000]
fields = generate_fields(fs, steps)
print(fields)

[140000, 135000, 130000, 125000, 120000, 115000, 110000, 105000, 100000, 95000, 90000, 85000, 80000, 75000, 70000, 65000, 60000, 55000, 50000, 45000, 40000, 35000, 30000, 25000, 20000, 15000, 10000, 5000, 0, -5000, -10000, -15000, -20000, -25000, -30000, -35000, -40000, -45000, -50000, -55000, -60000, -65000, -70000, -75000, -80000, -85000, -90000, -95000, -100000, -105000, -110000, -115000, -120000, -125000, -130000, -135000, -140000, -135000, -130000, -125000, -120000, -115000, -110000, -105000, -100000, -95000, -90000, -85000, -80000, -75000, -70000, -65000, -60000, -55000, -50000, -45000, -40000, -35000, -30000, -25000, -20000, -15000, -10000, -5000, 0, 5000, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000, 55000, 60000, 65000, 70000, 75000, 80000, 85000, 90000, 95000, 100000, 105000, 110000, 115000, 120000, 125000, 130000, 135000, 140000]


In [24]:
set_T(320)

CalledProcessError: Command '['C:\\Program Files\\IronPython 2.7\\ipy.exe', 'C:\\Users\\yanglabuser\\Desktop\\Sam files\\_switch\\ppms-master\\ppms.py', '320']' returned non-zero exit status 1.

In [34]:
do_task=True

In [35]:
if not do_task:
    raise
Ts = [330, 340]
wait_after_T = 15
field_scan(dc1, volt1, dc_current=0.0001,fields=fields, Ts=Ts, 
               sample_id="XD798_1_Al2O3_3nm_Pt_10nm_Mn3Sn_4nmSTO", t_wait=wait_after_T, field_wait=5)
set_Field(0, True)
set_T(300)
do_task=False

current T 330.0635
140000.09375
current field 140000.09375
134999.984375
current field 134999.984375
129999.460938
current field 129999.460938
124999.664063
current field 124999.664063
119999.84375
current field 119999.84375
114999.695313
current field 114999.695313
109999.570313
current field 109999.570313
104999.59375
current field 104999.59375
99999.59375
current field 99999.59375
94999.15625
current field 94999.15625
89999.578125
current field 89999.578125
84999.90625
current field 84999.90625
79999.8828125
current field 79999.8828125
74999.4140625
current field 74999.4140625
69999.7890625
current field 69999.7890625
64999.5429688
current field 64999.5429688
59999.84375
current field 59999.84375
54999.8476563
current field 54999.8476563
49999.6210938
current field 49999.6210938
44999.578125
current field 44999.578125
39999.828125
current field 39999.828125
34999.53125
current field 34999.53125
29999.6601563
current field 29999.6601563
24999.7636719
current field 24999.7636719
19999

-9999.8203125
current field -9999.8203125
-4999.84765625
current field -4999.84765625
5.75756153864e-09
current field 5.75756153864e-09
5000.02246094
current field 5000.02246094
9999.94433594
current field 9999.94433594
14999.9931641
current field 14999.9931641
20000.015625
current field 20000.015625
24999.9863281
current field 24999.9863281
30000.1367188
current field 30000.1367188
35000.1835938
current field 35000.1835938
40000.3320313
current field 40000.3320313
45000.2539063
current field 45000.2539063
50000.125
current field 50000.125
55000.046875
current field 55000.046875
59999.9453125
current field 59999.9453125
65000.1914063
current field 65000.1914063
69999.390625
current field 69999.390625
74999.8671875
current field 74999.8671875
80000.15625
current field 80000.15625
85000.1875
current field 85000.1875
90000.28125
current field 90000.28125
95000.3984375
current field 95000.3984375
100000.273438
current field 100000.273438
105000.570313
current field 105000.570313
109999.695

In [33]:

set_T(300)
#time.sleep(30*60)

current T 300.0475


In [41]:
set_T(300)

current T 300.069


In [ ]:
set_Field(0)

-5.20156895334e-09
current field -5.20156895334e-09


In [261]:
fields

[60000,
 55000,
 50000,
 45000,
 40000,
 35000,
 30000,
 25000,
 20000,
 18000,
 16000,
 14000,
 12000,
 10000,
 8000,
 6000,
 4000,
 2000,
 0,
 -2000,
 -4000,
 -6000,
 -8000,
 -10000,
 -12000,
 -14000,
 -16000,
 -18000,
 -20000,
 -25000,
 -30000,
 -35000,
 -40000,
 -45000,
 -50000,
 -55000,
 -60000,
 -55000,
 -50000,
 -45000,
 -40000,
 -35000,
 -30000,
 -25000,
 -20000,
 -18000,
 -16000,
 -14000,
 -12000,
 -10000,
 -8000,
 -6000,
 -4000,
 -2000,
 0,
 2000,
 4000,
 6000,
 8000,
 10000,
 12000,
 14000,
 16000,
 18000,
 20000,
 25000,
 30000,
 35000,
 40000,
 45000,
 50000,
 55000,
 60000]

In [55]:
set_Field(0)

0.0250031352043
current field 0.0250031352043


In [41]:
set_T(300)
set_Field(0)

current T 299.9995
-0.0250031296164
current field -0.0250031296164


In [66]:
do_task = True

In [15]:
fields = list(range(30000, 5000,-5000)) + list(range(5000,0,-500)) + [0]
print(fields)

[30000, 25000, 20000, 15000, 10000, 5000, 4500, 4000, 3500, 3000, 2500, 2000, 1500, 1000, 500, 0]


[0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 450, 400, 350, 300, 250, 200, 150, 100, 50, 0]


In [51]:
set_T(300)

current T 300.0092


In [79]:
do_task=True
test = False

In [80]:
if not do_task:
    raise

if test:
    fs = [1000]
    steps = [100]
    Ts = [None]
else:
    
    Ts = [375, 395]#[250, 275, 300, 325, 345]
    fs = [140000]
    steps = [5000]

fields = generate_fields(fs, steps)
print(fields)
field_scan_two_sample(dc1, volt1, dc2, volt2, dc_current_1=0.00005, dc_current_2=0.00005, fields=fields, Ts=Ts, 
                      sample1_id="N.A", 
                      sample2_id="XD801_1_Al2o3111_1p5nm_Pt_20nm_Mn3Sn_1p5nmTa_1nmPt", t_wait=15, field_wait=2)
set_Field(0, True)
set_T(300)
do_task = False

[140000, 135000, 130000, 125000, 120000, 115000, 110000, 105000, 100000, 95000, 90000, 85000, 80000, 75000, 70000, 65000, 60000, 55000, 50000, 45000, 40000, 35000, 30000, 25000, 20000, 15000, 10000, 5000, 0, -5000, -10000, -15000, -20000, -25000, -30000, -35000, -40000, -45000, -50000, -55000, -60000, -65000, -70000, -75000, -80000, -85000, -90000, -95000, -100000, -105000, -110000, -115000, -120000, -125000, -130000, -135000, -140000, -135000, -130000, -125000, -120000, -115000, -110000, -105000, -100000, -95000, -90000, -85000, -80000, -75000, -70000, -65000, -60000, -55000, -50000, -45000, -40000, -35000, -30000, -25000, -20000, -15000, -10000, -5000, 0, 5000, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000, 55000, 60000, 65000, 70000, 75000, 80000, 85000, 90000, 95000, 100000, 105000, 110000, 115000, 120000, 125000, 130000, 135000, 140000]
current T 375.1454
2019-12-23 12:00:15.767000
139999.65625
current field 139999.65625
134999.65625
current field 134999.65625
1299

-124999.414063
current field -124999.414063
-119999.390625
current field -119999.390625
-114998.765625
current field -114998.765625
-109999.820313
current field -109999.820313
-104999.796875
current field -104999.796875
-99999.71875
current field -99999.71875
-94999.8984375
current field -94999.8984375
-89999.4765625
current field -89999.4765625
-84999.484375
current field -84999.484375
-79999.7578125
current field -79999.7578125
-74999.9375
current field -74999.9375
-69999.7890625
current field -69999.7890625
-64999.7382813
current field -64999.7382813
-59999.921875
current field -59999.921875
-54999.9492188
current field -54999.9492188
-49999.7226563
current field -49999.7226563
-44999.5273438
current field -44999.5273438
-39999.6289063
current field -39999.6289063
-34999.8085938
current field -34999.8085938
-29999.7597656
current field -29999.7597656
-24999.7636719
current field -24999.7636719
-19999.7402344
current field -19999.7402344
-14999.6914063
current field -14999.6914063
-9

In [32]:
set_T(300)
set_Field(0,True)

current T 300.0477
-0.0233362391591
current field -0.0233362391591


In [76]:
fs = [80000, 20000]
steps = [10000,5000]

fields = generate_fields(fs, steps)
print(fields)

[80000, 70000, 60000, 50000, 40000, 30000, 20000, 15000, 10000, 5000, 0, -5000, -10000, -15000, -20000, -30000, -40000, -50000, -60000, -70000, -80000, -70000, -60000, -50000, -40000, -30000, -20000, -15000, -10000, -5000, 0, 5000, 10000, 15000, 20000, 30000, 40000, 50000, 60000, 70000, 80000]


In [86]:
do_task=True
test = False
back_to_300 = True

In [87]:
#lock in
if not do_task:
    raise

if test:
    fs = [100]
    steps = [50]
    fs = [5000]
    steps = [500]
    
    Ts = [None]
else:
    
    Ts = [None]
    fs = [50000, 10000]
    steps = [5000, 2000]

fields = generate_fields(fs, steps)[::-1]
fields = fields[:len(fields)//2]
print(fields)
field_scan_lock_in(lock, first_range=0.05, second_range=0.00002, time_step=0.1, input_I =1, wait_before_measure=30, 
                       count=150, fields=fields, Ts=Ts, sample1_id="Fe2O3_non_local_1p6um_1mA_10K_90degree_fake120", t_wait=15, field_wait=5)
if back_to_300:
    set_T(300)
    set_Field(0, True)
do_task = False

[50000, 45000, 40000, 35000, 30000, 25000, 20000, 15000, 10000, 8000, 6000, 4000, 2000, 0, -2000, -4000, -6000, -8000, -10000, -15000, -20000, -25000, -30000, -35000, -40000, -45000]
50000.2734375
current field 50000.2734375
phase 1
(0.009254437653968666, 0.028028058049560003, 0.029516379199906666, 71.72755813591999, 1.879662545214194e-07, 3.8200613881264503e-07, 3.979025543749098e-07, 0.00029528774590383407)
phase2
(-2.6536940079846666e-06, 2.465006114106e-06, 3.623786944144e-06, 137.08681376153336, 1.6024256443767327e-07, 1.1992686587512466e-07, 1.5676301533767113e-07, 1.9542022977723679)
44999.9296875
current field 44999.9296875
phase 1
(0.009254191561297333, 0.02802767125268667, 0.029515934723126663, 71.72777628578001, 1.7914869145292255e-07, 2.1114313747430605e-07, 2.489624660867798e-07, 0.00023415717390228694)
phase2
(-2.7867010491393334e-06, 2.746979722968667e-06, 3.917317550681334e-06, 135.44313415533333, 1.3433162601735156e-07, 1.8339626050921098e-07, 1.365426050228425e-07, 2.

(-2.9323430377813337e-06, 2.787562639244e-06, 4.047556224878667e-06, 136.4682817586, 1.226561924323816e-07, 1.5831273215158026e-07, 1.643625710279779e-07, 1.614009268768525)
-20000.3652344
current field -20000.3652344
phase 1
(0.009249765574931334, 0.028017981958900002, 0.029505346653406667, 71.73003921510666, 1.9817476865999647e-07, 1.8367465277063173e-07, 2.2033676796550482e-07, 0.00030247419075238096)
phase2
(-2.9066702791166664e-06, 2.542793942362e-06, 3.865370246483333e-06, 138.89311747213335, 1.4224293393722224e-07, 2.219654718382298e-07, 2.1023475136979666e-07, 2.3583364983173647)
-25000.2382813
current field -25000.2382813
phase 1
(0.009249405016502666, 0.028016148880119995, 0.029503492936546666, 71.72958831786669, 1.129864311792748e-07, 1.8361436644178925e-07, 1.85745628394314e-07, 0.0002131229731362458)
phase2
(-2.8658253161966667e-06, 2.6782902902286667e-06, 3.933650411154e-06, 136.90674316400003, 2.309823768500777e-07, 1.9239363684285212e-07, 7.732151869227076e-08, 4.238726

RuntimeError: No active exception to reraise

In [76]:
def lock_angular_measurement(lock, first_range=0.01, second_range=0.00001, time_step=0.1,
                             input_I =0.001, wait_before_measure=30, 
                             lock_count=10, sample_id="Fe2O3",  
                             time_per_point=1, time_out_count = 60*5):
    filename = "./_switch/data/" + sample_id + "_{}.csv".format(int(time.time()))
    with open(filename, 'w') as f:
        f.write("x,y,r,theta,std_x,std_y,std_r,std_theta,x2,y2,r2,theta2,std_x2,std_y2,std_r2,std_theta2,ac_I,POS\n")
    point_count = 0
    count = 0 # if position didn't change count++, stop measurement once count > 
    last_position = None
    while count < time_out_count:
        pos = get_position()
        print("current position: {}, count:{}".format(pos, count))
        if pos == last_position and count == 1:
            count += 1
            res1 = lock.lock_in_measure(lock_count, time_step, wait_before_measure, harm=1,sens=first_range)
            res2 = lock.lock_in_measure(lock_count, time_step, wait_before_measure, harm=2,sens=second_range)

            with open(filename, 'a') as f:
                f.write("{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(*res1, *res2, input_I, pos))
        elif pos == last_position:
            count += 1
        else:
            count =0
            last_position = pos
        time.sleep(time_per_point)
                

In [12]:
def lock_angular_measurement_set_p(positions, lock, first_range=0.01, second_range=0.00001, time_step=0.1,
                             input_I =0.001, wait_before_measure=30, 
                             lock_count=10, sample_id="Fe2O3",  
                             time_after_move=5, time_out_count = 60*5):
    filename = "./data/" + sample_id + "_{}.csv".format(int(time.time()))
    with open(filename, 'w') as f:
        f.write("x,y,r,theta,std_x,std_y,std_r,std_theta,x2,y2,r2,theta2,std_x2,std_y2,std_r2,std_theta2,ac_I,POS\n")
    
    for pos in positions:
        set_position(pos)
        time.sleep(time_after_move)
        print("current position: {}".format(pos))
        res1 = lock.lock_in_measure(lock_count, time_step, wait_before_measure, harm=1,sens=first_range)
        res2 = lock.lock_in_measure(lock_count, time_step, wait_before_measure, harm=2,sens=second_range)

        with open(filename, 'a') as f:
            f.write("{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(*res1, *res2, input_I, pos))
    set_position(0)
    time.sleep(8)

In [13]:
do_task = True

In [209]:
ps = list(range(-5, 15, 3)) + list(range(15, 165, 10)) + list(range(165, 195, 3)) + list(range(195, 346, 10))

In [14]:
ps = list(range(-5, 346, 10))

In [15]:
len(ps)

36

In [16]:
do_task = True

In [17]:
#rxy measurment 14, 10
fields = [10000, 40000, 80000, 100000]
temp = 300
#set_T(temp)
#time.sleep(25*60)
if not do_task:
    raise
for f in fields:
    set_Field(f, True)

    lock_angular_measurement_set_p(ps, lock, sample_id="2nmPt_Fe2o3_30nm_test_3mA_rxy_gamma"+ str(temp) + "K" + str(f)+ "G", 
                               first_range=0.01, second_range=0.00001,wait_before_measure=60)
     
set_Field(0, True)
do_task=False

9999.98339844
current field 9999.98339844
current position: -5
current position: 5
current position: 15
current position: 25
current position: 35
current position: 45
current position: 55
current position: 65
current position: 75
current position: 85
current position: 95
current position: 105
current position: 115
current position: 125
current position: 135
current position: 145
current position: 155
current position: 165
current position: 175
current position: 185
current position: 195
current position: 205
current position: 215
current position: 225
current position: 235
current position: 245
current position: 255
current position: 265
current position: 275
current position: 285
current position: 295
current position: 305
current position: 315
current position: 325
current position: 335
current position: 345
40000.453125
current field 40000.453125
current position: -5
current position: 5
current position: 15
current position: 25
current position: 35
current position: 45
current posit

In [77]:
do_task=True

In [190]:
set_Field(0, True)
set_T(300)

-0.000833437079564
current field -0.000833437079564
current T 300.0305


In [82]:
if not do_task:
    raise 
lock_angular_measurement(lock, first_range=0.05, second_range=0.00002, time_step=0.1,
                             input_I =0.001, wait_before_measure=30, 
                             lock_count=150, sample_id="XD531_Fe2O3_10K_1mA_17Hz_redo",  
                             time_per_point=1, time_out_count = 60*4)
#set_Field(0, True)
#set_T(100)
do_task=False

current position: 0, count:0
current position: 0, count:0
current position: 0, count:1
current position: 0, count:2
current position: 0, count:3
current position: 0, count:4
current position: 0, count:5
current position: 0, count:6
current position: 0, count:7
current position: 6, count:8
current position: 10, count:0
current position: 10, count:0
current position: 10, count:1
current position: 10, count:2
current position: 10, count:3
current position: 10, count:4
current position: 10, count:5
current position: 10, count:6
current position: 10, count:7
current position: 10, count:8
current position: 10, count:9
current position: 10, count:10
current position: 16, count:11
current position: 20, count:0
current position: 20, count:0
current position: 20, count:1
current position: 20, count:2
current position: 20, count:3
current position: 20, count:4
current position: 20, count:5
current position: 20, count:6
current position: 20, count:7
current position: 20, count:8
current position: 

current position: 190, count:10
current position: 196, count:11
current position: 200, count:0
current position: 200, count:0
current position: 200, count:1
current position: 200, count:2
current position: 200, count:3
current position: 200, count:4
current position: 200, count:5
current position: 200, count:6
current position: 200, count:7
current position: 200, count:8
current position: 200, count:9
current position: 200, count:10
current position: 204, count:11
current position: 210, count:0
current position: 210, count:0
current position: 210, count:1
current position: 210, count:2
current position: 210, count:3
current position: 210, count:4
current position: 210, count:5
current position: 210, count:6
current position: 210, count:7
current position: 210, count:8
current position: 210, count:9
current position: 210, count:10
current position: 211, count:11
current position: 218, count:0
current position: 220, count:0
current position: 220, count:0
current position: 220, count:1
cu

current position: 22, count:0
current position: 10, count:0
current position: -2, count:0
current position: -3, count:0
current position: -3, count:0
current position: -3, count:1
current position: -3, count:2
current position: -3, count:3
current position: -3, count:4
current position: -3, count:5
current position: -3, count:6
current position: -3, count:7
current position: -3, count:8
current position: -3, count:9
current position: -3, count:10
current position: -3, count:11
current position: -3, count:12
current position: -3, count:13
current position: -3, count:14
current position: -3, count:15
current position: -3, count:16
current position: -3, count:17
current position: -3, count:18
current position: -3, count:19
current position: -3, count:20
current position: -3, count:21
current position: -3, count:22
current position: -3, count:23
current position: -3, count:24
current position: -3, count:25
current position: -3, count:26
current position: -3, count:27
current position: -3, 

In [ ]:
set_T(300)
set_Field(0, True)

current T 299.99
-0.00208359281532
current field -0.00208359281532


In [40]:
def fm_switch(ac, dc, volt, currents, dc_current, duration, file_name):
    file_name = "data/{}_FM_SWITCH_{}.csv".format(file_name, int(time.time()))
    with open(file_name, 'w') as f:
        f.write("pulse,v,std_v,I\n")
    for c in currents:
        trigger_ac(ac, c,duration=duration)
        time.sleep(10)
        average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
        with open(file_name, 'a') as f:
            f.write("{},{},{},{}\n".format(c, average_v, std_v, dc_current))
                    

In [44]:
maxI = 19
i = list(range(-1* maxI, maxI, 1)) + list(range(maxI, -1 * (maxI + 1), -1))
i = [m*0.001 for m in i]
i = [float(str(m)[:6]) for m in i]
currents = i
print(currents)
do_task=True

[-0.019, -0.018, -0.017, -0.016, -0.015, -0.014, -0.013, -0.012, -0.011, -0.01, -0.009, -0.008, -0.007, -0.006, -0.005, -0.004, -0.003, -0.002, -0.001, 0.0, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017, 0.018, 0.019, 0.018, 0.017, 0.016, 0.015, 0.014, 0.013, 0.012, 0.011, 0.01, 0.009, 0.008, 0.007, 0.006, 0.005, 0.004, 0.003, 0.002, 0.001, 0.0, -0.001, -0.002, -0.003, -0.004, -0.005, -0.006, -0.007, -0.008, -0.009, -0.01, -0.011, -0.012, -0.013, -0.014, -0.015, -0.016, -0.017, -0.018, -0.019]


In [45]:
tmp = []
for i in currents:
    tmp += [i] * 5
currents = tmp
print(currents)

[-0.019, -0.019, -0.019, -0.019, -0.019, -0.018, -0.018, -0.018, -0.018, -0.018, -0.017, -0.017, -0.017, -0.017, -0.017, -0.016, -0.016, -0.016, -0.016, -0.016, -0.015, -0.015, -0.015, -0.015, -0.015, -0.014, -0.014, -0.014, -0.014, -0.014, -0.013, -0.013, -0.013, -0.013, -0.013, -0.012, -0.012, -0.012, -0.012, -0.012, -0.011, -0.011, -0.011, -0.011, -0.011, -0.01, -0.01, -0.01, -0.01, -0.01, -0.009, -0.009, -0.009, -0.009, -0.009, -0.008, -0.008, -0.008, -0.008, -0.008, -0.007, -0.007, -0.007, -0.007, -0.007, -0.006, -0.006, -0.006, -0.006, -0.006, -0.005, -0.005, -0.005, -0.005, -0.005, -0.004, -0.004, -0.004, -0.004, -0.004, -0.003, -0.003, -0.003, -0.003, -0.003, -0.002, -0.002, -0.002, -0.002, -0.002, -0.001, -0.001, -0.001, -0.001, -0.001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001, 0.001, 0.001, 0.001, 0.001, 0.002, 0.002, 0.002, 0.002, 0.002, 0.003, 0.003, 0.003, 0.003, 0.003, 0.004, 0.004, 0.004, 0.004, 0.004, 0.005, 0.005, 0.005, 0.005, 0.005, 0.006, 0.006, 0.006, 0.006, 0.006, 0.007, 0.

In [95]:

#switch
ac_6221_1_id = 14
#ac_6221__3id = 12 #mid
dc_2400_id = 24
nanovolt_id = 9

ac_1 = rm.open_resource('GPIB0::{}::INSTR'.format(ac_6221_1_id))
dc = Keithley2400("GPIB::{}".format(dc_2400_id))
volt = rm.open_resource('GPIB0::{}::INSTR'.format(nanovolt_id))

In [46]:
if not do_task:
    raise 
fm_switch(ac_2, dc, volt, currents, 0.0001, 0.001, "D8075nmMn3Sn_test_FM_switch")
do_task = False

In [35]:
i = list(range(-20, 20, 1)) + list(range(20, -21, -1))
i = [m*0.001 for m in i]
i = [float(str(m)[:6]) for m in i]

In [36]:
print(i)

[-0.02, -0.019, -0.018, -0.017, -0.016, -0.015, -0.014, -0.013, -0.012, -0.011, -0.01, -0.009, -0.008, -0.007, -0.006, -0.005, -0.004, -0.003, -0.002, -0.001, 0.0, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017, 0.018, 0.019, 0.02, 0.019, 0.018, 0.017, 0.016, 0.015, 0.014, 0.013, 0.012, 0.011, 0.01, 0.009, 0.008, 0.007, 0.006, 0.005, 0.004, 0.003, 0.002, 0.001, 0.0, -0.001, -0.002, -0.003, -0.004, -0.005, -0.006, -0.007, -0.008, -0.009, -0.01, -0.011, -0.012, -0.013, -0.014, -0.015, -0.016, -0.017, -0.018, -0.019, -0.02]
